In [ ]:
!pip install -U spacy==3.0

     |████████████████████████████████| 12.7 MB 4.0 MB/s 
     |████████████████████████████████| 628 kB 38.6 MB/s 
     |████████████████████████████████| 451 kB 49.8 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 9.1 MB 14.7 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [ ]:
import spacy
import json
from spacy.matcher import Matcher
from spacy.lang.es import Spanish
from spacy.lang.en import English
from spacy.lang.xx import MultiLanguage
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher
import random
from spacy.util import minibatch, compounding
from spacy.training.example import Example
# from spacy.training.example import Example
import pandas as pd
import re

In [12]:
from google.colab import drive
drive.mount('/content/drive')

ruta = '../content/drive/My Drive/list.csv'

xls = pd.read_csv(ruta)

frases = xls.corpus.tolist()

comida = ["canelones","croquetas","patatas","pollo", "cerdo asado", "cordero al horno", "carrilleras", "solomillo", "pechuga", "sopa", "sushi", "ramen", "rollitos primavera","galletas", "pato", "cookies", "oreo", "chocolate", "turron", "tarta", "roscon", "pure", "verduras", "sofrito", "merluza", "pescado", "rape", "pasta", "macarrones", "espagueti", "tomate", "natillas", "profiterol", "paella", "arroz", "fideua", "vichyssoise"]
personas = ["raul", "alex", "alvaro", "tere", "ada", "gaspar", "francisco", "ian", "tomas", "james", "laura", "maria", "isaac", "carlos", "andres", "javier", "luis", "carmen", "gonzalo","catalina", "juana", "natalia", "lucia", "eva", "marc", "jorge", "ernesto", "teresa", "marcelino", "maite", "monica", "toni", "victor", "meritxell", "blanca", "pilar", "paula"]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

os.system("python -m spacy download en_core_web_sm")
os.system("python -m spacy download en_core_web_md")

0

#Cargando un modelo entrenado

In [ ]:
nlp = spacy.load("en_core_web_sm")

text = "It’s official: Apple is the first U.S. public company to reach a $1 trillion market value"

# Process the text
doc = nlp(text)

for token in doc:
    # Get the token text, part-of-speech tag and dependency label
    token_text = token.text
    token_pos = token.pos_
    token_dep = token.dep_
    # This is for formatting only
    print(f"{token_text:<12}{token_pos:<10}{token_dep:<10}")

It          PRON      nsubj     
’s          VERB      ccomp     
official    ADJ       acomp     
:           PUNCT     punct     
Apple       PROPN     nsubj     
is          AUX       ROOT      
the         DET       det       
first       ADJ       amod      
U.S.        PROPN     nmod      
public      ADJ       amod      
company     NOUN      attr      
to          PART      aux       
reach       VERB      relcl     
a           DET       det       
$           SYM       quantmod  
1           NUM       compound  
trillion    NUM       nummod    
market      NOUN      compound  
value       NOUN      dobj      


 #Matcher

In [ ]:
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
doc = nlp("Today we are in the University of Barcelona with the GDSC UB")

# Initialize the Matcher with the shared vocabulary
matcher = Matcher(nlp.vocab)

# Create a pattern matching two tokens: "iPhone" and "X"
pattern = [{"TEXT": "GDSC"}, {"TEXT": "UB"}]

# Add the pattern to the matcher
matcher.add("GDSCUB_PATTERN", [pattern])

# Use the matcher on the doc
matches = matcher(doc)
print("Matches:", [doc[start:end].text for match_id, start, end in matches])

Matches: ['GDSC UB']


#Similarity

In [ ]:
nlp = spacy.load("en_core_web_md")

doc1 = nlp("It's a warm summer day")
doc2 = nlp("It's sunny outside")

# Get the similarity of doc1 and doc2
similarity = doc1.similarity(doc2)
print(similarity)

0.8789265574516525


#NER con redes entrenadas

In [ ]:
doc = nlp(
    "The event of the GDSC lead by Ian Carlos is awesome"
)

for ent in doc.ents:
    # Print the text and Wikipedia URL of the entity
    print(ent.label_,ent.text)

ORG GDSC
PERSON Ian Carlos


#Entrenando un modelo desde cero

In [13]:

nlp = Spanish()
matcher = PhraseMatcher(nlp.vocab)
matcher.add("Persona", None, *list(nlp.pipe(personas)))
matcher.add("comida", None, *list(nlp.pipe(comida)))

In [14]:
training_data = []
for doc in nlp.pipe(frases):
    matches = matcher(doc)
    entities = []
    for match_id, match_start, match_end in matches:
      string_id = nlp.vocab.strings[match_id]  # Get string representation
      span = doc[match_start:match_end]  # The matched span
      entity = [(span.start_char, span.end_char, string_id)][0]
      entities.append(entity)
    training_example = (doc.text, {"entities": entities})
    training_data.append(training_example)

In [15]:
nlp = spacy.blank("es")  # create blank Language class
print("Created blank 'es' model")

#Añadimos el reconocimiento de entidades al model si no está ya en el pipeline
#nlp.create_pipe works for built-ins that are registered with spaCy
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe("ner")
# otherwise, get it, so we can add labels to it
else:
    ner = nlp.get_pipe("ner")

Created blank 'es' model


In [16]:
ner.add_label("Persona") 
ner.add_label("comida")

1

In [17]:
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [18]:
TRAINING_DATA = training_data
nlp.begin_training()

optimizer = nlp.resume_training()
with nlp.disable_pipes(*other_pipes):  # only train NER
    sizes = compounding(1.0, 4.0, 1.001)

from tqdm import tqdm
for itn in tqdm(range(10)):
    # shuffle examples before training
    random.shuffle(TRAINING_DATA)
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAINING_DATA, size=sizes)
    #dictionary to store losses
    losses = {}
    for batch in batches:
      #print(batch)
      for text, annotations in batch:
        doc = nlp.make_doc(text)
        #print(text)
        example = Example.from_dict(doc, annotations)
      # Calling update() over the iteration
        nlp.update([example], sgd=optimizer, drop=0.50, losses=losses)

  0%|          | 0/10 [00:00<?, ?it/s]

Se han truncado las últimas 5000 líneas del flujo de salida.
meritxell comera pato en un restaurante
gonzalo come merluza
ian esta comiendo roscon
victor comera arroz en un restaurante
a lucia le apetece arroz
teresa cocinara chocolate
maite cocinara cookies
a eva le apetece comer verduras
alvaro come tomate
a james le apetece comer natillas
ada cocinara tomate
carlos va a preparar macarrones
toni esta cocinando macarrones
meritxell esta comiendo rollitos primavera
a paula le apetece fideua
meritxell quiere comer patatas
juana quiere comer espagueti
a gonzalo le apetece natillas
a luis le apetece ramen
catalina esta comiendo tarta
carlos cocinara sushi
gaspar comera sofrito en un restaurante
meritxell va a preparar turron
raul cocinara galletas
alvaro cocinara pure
luis quiere comer solomillo
carlos quiere comer patatas
a raul le apetece comer arroz
meritxell quiere comer tarta
andres cocinara pasta
ada va a preparar patatas
javier cocinara pescado
blanca esta comiendo canelones
a paul

 10%|█         | 1/10 [02:53<26:04, 173.82s/it]

Se han truncado las últimas 5000 líneas del flujo de salida.
eva quiere comer oreo
a pilar le apetece pechuga
eva quiere comer patatas
maite come canelones
maria cocinara vichyssoise
meritxell come tarta
paula comera patatas en un restaurante
maria esta comiendo rape
a alvaro le apetece fideua
maite come chocolate
a tomas le apetece comer carrilleras
isaac esta comiendo sofrito
carmen quiere comer sofrito
para comer tenemos profiterol
jorge come sushi
isaac quiere comer pollo
francisco esta comiendo canelones
marc come pasta
a ernesto le apetece pasta
carlos esta cocinando rape
alex esta cocinando merluza
gonzalo quiere comer cerdo asado
gonzalo esta cocinando sopa
gonzalo esta cocinando roscon
lucia quiere comer rape
luis esta cocinando tarta
marc come natillas
a carlos le apetece arroz
marcelino esta cocinando patatas
tere esta comiendo solomillo
victor come patatas
luis esta cocinando pollo
ian va a preparar oreo
pilar come arroz
a carmen le apetece comer pasta
eva comera fideua en 

 20%|██        | 2/10 [06:33<26:47, 200.92s/it]

...

Se han truncado las últimas 5000 líneas del flujo de salida.
a jorge le apetece espagueti
jorge va a preparar ramen
carmen cocinara rollitos primavera
monica esta cocinando canelones
blanca come profiterol
carmen esta comiendo solomillo
pilar esta comiendo pechuga
toni come sofrito
a maite le apetece comer verduras
carmen esta cocinando canelones
marc come sopa
ian esta comiendo oreo
natalia cocinara arroz
a isaac le apetece turron
a lucia le apetece comer tomate
ian come oreo
toni va a preparar rape
victor va a preparar macarrones
carlos va a preparar espagueti
francisco va a preparar arroz
ernesto quiere comer patatas
victor esta cocinando verduras
teresa comera profiterol en un restaurante
raul comera patatas en un restaurante
javier quiere comer pato
blanca esta comiendo oreo
a alvaro le apetece comer pure
blanca cocinara fideua
gonzalo come pato
ian come rape
ada comera patatas en un restaurante
monica cocinara canelones
carlos esta cocinando sopa
maite cocinara arroz
maria cocina

 90%|█████████ | 9/10 [33:24<03:46, 226.02s/it]

Se han truncado las últimas 5000 líneas del flujo de salida.
laura comera cordero al horno en un restaurante
maria comera tomate en un restaurante
a jorge le apetece vichyssoise
tere esta comiendo espagueti
gaspar va a preparar rollitos primavera
tomas comera pasta en un restaurante
a marcelino le apetece espagueti
francisco va a preparar natillas
catalina esta comiendo pescado
andres comera arroz en un restaurante
a blanca le apetece comer patatas
carmen quiere comer vichyssoise
tomas esta cocinando pato
natalia esta comiendo patatas
luis cocinara verduras
carlos quiere comer pure
toni esta cocinando patatas
a andres le apetece rape
a alvaro le apetece comer pasta
carlos come turron
maite esta comiendo oreo
a marcelino le apetece rape
toni comera patatas en un restaurante
monica comera ramen en un restaurante
isaac esta comiendo vichyssoise
tere va a preparar tarta
a tomas le apetece cookies
a maria le apetece rape
james va a preparar tomate
lucia quiere comer ramen
a maite le apetece

100%|██████████| 10/10 [37:18<00:00, 223.84s/it]

pilar quiere comer cookies
lucia esta cocinando ramen
tomas comera chocolate en un restaurante
a alex le apetece espagueti
a alex le apetece pescado


In [20]:
test_text = "Alex esta comiendo queso"
doc = nlp(test_text)
print("Entities in '%s'" % test_text)
for ent in doc.ents:
    print(ent.label_, ent.text)

Entities in 'Alex esta comiendo queso'
Persona Alex
comida queso


In [21]:
from pathlib import Path
# save model to output directory
output_dir = "../content/drive/My Drive//NER_prueba"
new_model_name = "ner_prueba"
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.meta["name"] = new_model_name  # rename model
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to ../content/drive/My Drive/NER_prueba


In [22]:
model_directory = "../content/drive/My Drive//NER_prueba"
model_saved = spacy.load(model_directory)

In [23]:
doc2 = model_saved("estas comiendo queso")
for ent in doc2.ents:
    entidad_label = ent.label_
    entidad_texto = ent.text
    print(entidad_label, entidad_texto)

Persona estas
comida queso
